# NSQIP Data Cleaning

## Notebook setup

Import pre-installed packages 

In [1]:
import pandas as pd
from pathlib import Path
from fastai.tabular.all import *

Package versions: 
* *Python v. 3.7.7*<br>
* *Pandas v. 1.1.4*<br>
* *PyTorch v. 1.6.0*<br>
* *Fastai v. 2.0.11*

Set display options

In [2]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = None

Set seed for reproducable results

In [3]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

Create a path to the NSQIP data folder

In [4]:
data_folder = Path(r'E:\Data\NSQIP')

## Data import and merging

### Get and merge NSQIP data from 2013-2018

Create a list of variables to import

In [5]:
var_col = ['CaseID', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'PRNCPTX', 'CPT', 'WORKRVU', 'Age', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'DOpertoD', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'SEPSHOCKPATOS', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'MORTPROB', 'MORBPROB', 'INOUT', 'TRANST', 'DPRNA', 'DPRBUN', 'DPRCREAT', 'DPRALBUM', 'DPRBILI', 'DPRSGOT', 'DPRALKPH', 'DPRWBC', 'DPRHCT', 'DPRPLATE', 'DPRPTT', 'DPRPT', 'DPRINR', 'OperYR', 'OPTIME', 'HtoODay', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME']

Specify integer variables, float variables and set the rest as string variables. 
Create a dictionary with variables + types. 

In [6]:
int_variables= ['CaseID', 'OperYR', 'HEIGHT', 'WEIGHT', 'DPRNA', 'DPRBUN', 'DPRCREAT', 'DPRALBUM', 'DPRBILI', 'DPRSGOT', 'DPRALKPH', 'DPRWBC', 'DPRHCT', 'DPRPLATE','DPRPTT', 'DPRPT', 'DPRINR']

In [7]:
float_variables=['PRSODM', 'PRBUN','PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT','WORKRVU', 'MORTPROB', 'MORBPROB', 'HtoODay', 'DOpertoD', 'OPTIME']

In [8]:
types_dict = {} 

In [9]:
for i in int_variables:
        types_dict[i] = int

In [10]:
for i in float_variables:
        types_dict[i] = float

In [11]:
types_dict.update({col: str for col in var_col if col not in types_dict})

Import data from 2013 to 2018. Some column names in the 2012 dataset are formatted differently, so 2012 data is handled seperately. 

In [12]:
file_name = 'acs_nsqip_puf{}.txt'
df_list = []
for i in range(13,19):
    df_list.append(pd.read_csv(data_folder/file_name.format(i), delimiter = '\t', usecols=var_col, dtype=types_dict))
    
nsqip_2013_2018 = pd.concat(df_list)

###  Get NSQIP data from 2012

All column names are capitalized in data from 2012: 

In [13]:
var_col_2012=[x.upper() for x in var_col]

In [14]:
int_variables_2012=[x.upper() for x in int_variables]

In [15]:
float_variables_2012=[x.upper() for x in float_variables]

In [16]:
types_dict_2012 = {} 

In [17]:
for i in int_variables_2012:
        types_dict_2012[i] = int

In [18]:
for i in float_variables_2012:
        types_dict_2012[i] = float

In [19]:
types_dict_2012.update({col: str for col in var_col_2012 if col not in types_dict_2012})

In [20]:
nsqip_2012 = pd.read_csv(data_folder/'acs_nsqip_puf12.txt', delimiter = '\t', usecols=var_col_2012, dtype=types_dict_2012)

Convert column names to match 2013-2018 data

In [21]:
nsqip_2012=nsqip_2012.rename(columns={"AGE": "Age", "DOPERTOD": "DOpertoD", "CASEID": "CaseID", 'OPERYR': 'OperYR',
                                      'HTOODAY': 'HtoODay'})

### Merge NSQIP data from 2012 with 2013-2018

In [22]:
nsqip = pd.concat([nsqip_2012, nsqip_2013_2018])

### Save NSQIP as feather format

In [24]:
nsqip.reset_index(inplace=True)

In [25]:
nsqip.to_feather(data_folder/'nsqip')

## Data cleaning

In [26]:
nsqip = pd.read_feather(data_folder/'nsqip')

### Replace missing values with NaN

Replace missing values with np.nan

In [27]:
nsqip["Age_cont"] = nsqip.Age.replace('90+', '90').astype(int)

In [28]:
nsqip=nsqip.replace(-99, np.nan)
nsqip=nsqip.replace('Unknown', np.nan)
nsqip=nsqip.replace('None assigned', np.nan)
nsqip=nsqip.replace('Unknown/Not Reported', np.nan)

### Handle categorical variables

*Ordinal columns;* Columns that contain strings with a natural ordering

In [29]:
def ordinal(column, sizes):
    nsqip[column] = nsqip[column].astype('category')
    return nsqip[column].cat.set_categories(sizes, ordered=True, inplace=True)

In [30]:
ordinal('DIABETES', ('NO','NON-INSULIN', 'INSULIN'))

In [31]:
ordinal('FNSTATUS2', ('Independent','Partially Dependent','Totally Dependent'))

In [32]:
ordinal('ASACLAS', ('1-No Disturb','2-Mild Disturb','3-Severe Disturb','4-Life Threat','5-Moribund'))

In [33]:
ordinal('PRSEPIS', ('None', 'SIRS', 'Sepsis', 'Septic Shock'))

In [34]:
ordinal('DYSPNEA', ('No', 'MODERATE EXERTION', 'AT REST'))

In [35]:
ordinal('WNDCLAS', ('1-Clean','2-Clean/Contaminated','3-Contaminated','4-Dirty/Infected'))

### Handle dependent variables

Create one-hot encoded columns for each of the 19 dependent variables. Some of the patients who had DVT where coded as *"DVT Requiring Therapy"* while others where coded as *"DVT Requiring Therap"*

In [36]:
def multi_label(df):
    SSSI,DSSI,OSSI,WOUND,PNEUMONIA,UNPINT,PE,VENT48,PRI,ARF,UTI,STROKE,CAR,MI,DVT1,DVT2,SEPSIS,SEPSHOCK,BLEED, DECEASED  = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    for row in df.itertuples():
        SSSI.append((row.SUPINFEC == 'Superficial Incisional SSI') & (row.SSSIPATOS != 'Yes')) 
        DSSI.append((row.WNDINFD == 'Deep Incisional SSI') & (row.DSSIPATOS != 'Yes')) 
        OSSI.append((row.ORGSPCSSI == 'Organ/Space SSI') & (row.OSSIPATOS != 'Yes')) 
        WOUND.append(row.DEHIS == 'Wound Disruption')
        PNEUMONIA.append((row.OUPNEUMO == 'Pneumonia') & (row.PNAPATOS != 'Yes'))
        UNPINT.append(row.REINTUB == 'Unplanned Intubation') 
        PE.append(row.PULEMBOL == 'Pulmonary Embolism')
        VENT48.append((row.FAILWEAN == 'On Ventilator greater than 48 Hours') & (row.VENTPATOS != 'Yes'))
        PRI.append(row.RENAINSF == 'Progressive Renal Insufficiency')
        ARF.append(row.OPRENAFL == 'Acute Renal Failure')
        UTI.append((row.URNINFEC == 'Urinary Tract Infection') & (row.UTIPATOS != 'Yes'))
        STROKE.append(row.CNSCVA == 'Stroke/CVA')
        CAR.append(row.CDARREST == 'Cardiac Arrest Requiring CPR')
        MI.append(row.CDMI == 'Myocardial Infarction')
        DVT1.append(row.OTHDVT == 'DVT Requiring Therapy')
        DVT2.append(row.OTHDVT == 'DVT Requiring Therap')
        SEPSIS.append((row.OTHSYSEP == 'Sepsis') & (row.SEPSISPATOS != 'Yes'))
        SEPSHOCK.append((row.OTHSESHOCK == 'Septic Shock') & (row.SEPSHOCKPATOS != 'Yes'))
        BLEED.append(row.OTHBLEED == 'Transfusions/Intraop/Postop')
        DECEASED.append(row.DOpertoD > -99)
    df['SSSI'] = np.array(SSSI)
    df['DSSI'] = np.array(DSSI)
    df['OSSI'] = np.array(OSSI)
    df['WOUND'] = np.array(WOUND)
    df['PNEUMONIA'] = np.array(PNEUMONIA)
    df['UNPINT'] = np.array(UNPINT)
    df['PE'] = np.array(PE)
    df['VENT48'] = np.array(VENT48)
    df['PRI'] = np.array(PRI)
    df['ARF'] = np.array(ARF)
    df['UTI'] = np.array(UTI)
    df['STROKE'] = np.array(STROKE)
    df['CAR'] = np.array(CAR)
    df['MI'] = np.array(MI)
    df['DVT1'] = np.array(DVT1)
    df['DVT2'] = np.array(DVT2)
    df['SEPSIS'] = np.array(SEPSIS)
    df['SEPSHOCK'] = np.array(SEPSHOCK)
    df['BLEED'] = np.array(BLEED)
    df['DECEASED'] = np.array(DECEASED)
    return df

In [37]:
nsqip = multi_label(nsqip)

Merge DVT columns

In [38]:
nsqip['DVT'] = nsqip.DVT1 | nsqip.DVT2 

### Save cleaned dataframe 

In [39]:
nsqip.to_feather(data_folder/'nsqip')